# <font style="color: rgb(0,0,0);">Joined Model</font> <font style="color: rgb(190,138,67);">Relevance Comparison</font>

In [9]:
# imports
import numpy as np
import xarray as xr

In [17]:
def info(dataset):
    print(f'{dataset.dtype} {dataset.shape} data range: {np.min(dataset)} - {np.max(dataset)}')

In [18]:
# A Result
ISRU_data = xr.open_dataarray("maps/ISRU_COST_GLOBAL.nc").data
info(ISRU_data)

float32 (13680, 27360) data range: 17054.625 - 157718.5


In [20]:
# B Results
cost_map_NRHO_1_25 = xr.open_dataarray("maps/cost_map_NRHO_1_25.nc").data
cost_map_NRHO_1_5  = xr.open_dataarray("maps/cost_map_NRHO_1_5.nc").data
info(cost_map_NRHO_1_25)
info(cost_map_NRHO_1_5)

float32 (11520, 23040) data range: 3.1104774475097656 - 3.199880599975586
float32 (11520, 23040) data range: 2.693203926086426 - 2.75943660736084
